In [ ]:
import pandas as pd
import numpy as np

In [ ]:
metadata_url = "https://mikeanders.org/data/CMS/CMS-2018-0101-0001/CMS-1701-P%20Comment%20MetaData.csv"
data = pd.read_csv(metadata_url, usecols=range(0,36))[:469] #ignore last few columns and blank rows at end of csv 
data = data.rename(columns=lambda x: x.strip()) #strip whitespace from columns

In [ ]:
data10 = data.fillna(0) #fill NaN with 0

In [ ]:
section_cols = data10.columns[3:] 
data10[section_cols] = data10[section_cols].replace(["Y"], 1) #replace Y with 1 in approriate columns

In [ ]:
section_cols


In [ ]:
data11 = data10.copy()

In [ ]:
section_cols1  = data11.columns[3:] 

In [ ]:
data11[section_cols1] = np.where((data11[section_cols1]  != 1),0,data11[section_cols1] )
data11

In [ ]:
# Combining columns for index matching: (A6b, A6b.1, = A6b),  (C3b, C3b.1'= C3b) ('A7', 'A7.1', 'A7.2', = A7b, a7c),  (F = F2, F3)

data11['A6b'] = (data11['A6b'] + data11['A6b.1'])
data11['A6b'] = data11['A6b'].replace(2,1)
data11['A6b'].value_counts()

In [ ]:
data11['C3b'] = (data11['C3b'] + data11['C3b.1'])
data11['C3b'] = data11['C3b'].replace(2,1)
data11['C3b'].value_counts()

In [ ]:
data11['A7'] = (data11['A7'] + data11['A7.1'] + data11['A7.2'])
data11['A7'] = data11['A7'].replace(2,1)
data11['A7'] = data11['A7'].replace(3,1)
data11['A7'].value_counts()

In [ ]:
data11 = data11.drop(['A6b.1', 'C3b.1', 'A7.1', 'A7.2'], axis = 1)

In [20]:
section_cols1  = data11.columns[3:]
section_cols1

Index(['A2', 'A3', 'A4b', 'A4c', 'A5b', 'A5c', 'A5d', 'A6b', 'A6c', 'A6d2',
       'A6d3', 'A7', 'B2a', 'B2b', 'C2', 'C3a', 'C3b', 'D2', 'D3b', 'D3c',
       'D3d', 'D4', 'E2', 'E3', 'E4', 'E5', 'E6', 'E7', 'F'],
      dtype='object')

In [ ]:
truth_dict = {} # initiate dict
for i, name in enumerate(data11.Name): # iterate through Comment Names
    dict1 = {col:data11[col].iloc[i] for col in section_cols1} # locate this row's values for each column
    list1 = [key.lower() for key,value in dict1.items() if value==1] # create list of matching sections for this row
    name1 = str(name).split('DRAFT-')[1].split('-')[0] # format Comment Name
    truth_dict[name1] = list1 # add Comment Name and list of matching sections to dict

In [ ]:
truth_dict

In [ ]:
num_matches = {key: len(value) for key, value in truth_dict.items()}
num_matches

## Header Index Testing

In [26]:
import json
with open('header_results.json') as f:
    header_results_json = json.load(f)
# comment_results1 = pd.read_csv('header_results.csv', header = None)
# comment_json = json.load('header_results.json')

In [24]:
for key,value in header_results_json.items():
    value_list = []
    for val_idx in value:
        if val_idx in ['f2','f3']:
            value_list.append('f')
        if val_idx in ['a7b', 'a7c']:
            value_list.append('a7')
        else:
            value_list.append(val_idx)
    header_results_json[key]=value_list

In [ ]:
header_results_json

In [ ]:
# match_dict shows all true positives, without penalty to false categorization
match_dict= {}
for (key1, value1), (key2,value2) in zip(truth_dict.items(), header_results_json.items()):
    match_count = 0
    for val_idx in value1:
        if val_idx in value2:
            match_count += 1
        match_dict[key1]=match_count/len(value1)

In [ ]:
# unpenalized accuracy of true positives
meanvals = np.mean(list(match_dict.values()))
meanvals

In [ ]:
# match_dict1 shows all true positives, with penalty to false categorization
match_dict1= {}
for (key1, value1), (key2,value2) in zip(truth_dict.items(), comment_json.items()):
    match_count = 0
    for val_idx in value2:
        if val_idx in value1:
            match_count += 1
        match_dict1[key1]=match_count/len(value2)

In [ ]:
# penalized accuracy of true positives
meanvals1 = np.mean(list(match_dict1.values()))
meanvals1

## Hybrid Index Testing

In [ ]:
import json
with open('hybrid_results.json') as f:
    hybrid_results_json = json.load(f)
# comment_results1 = pd.read_csv('header_results.csv', header = None)
# comment_json = json.load('header_results.json')

In [34]:
test = {}
for key, value in header_results_json.items():
    keys = list(section_cols1)
    keys = [key1.lower() for key1 in keys]
    value_list = []
    for true_key in keys:
        for val_idx in value:
            if true_key in val_idx and true_key not in value_list: 
                value_list.append(true_key)
        test[key]=value_list            

In [ ]:
test

In [ ]:
# for key,value in hybrid_results_json.items():
#     value_list = []
#     for val_idx in value:
#         if val_idx in ['a2','a20']:
#             value_list.append('a2')
#         if val_idx in ['a3','a30','a31','a32']:
#             value_list.append('a3')
#         if val_idx in ['a4b','a4b0']:
#             value_list.append('a4b')     
#         if val_idx in ['a4c','a4c0','a4c1','a4c10','a4c11', 'a4c12','a4c13','a4c14',
#                        'a4c15','a4c16','a4c17','a4c18','a4c19','a4c2','a4c20','a4c21',
#                        'a4c22','a4c23','a4c24','a4c25','a4c26','a4c27','a4c28','a4c29',
#                        'a4c3','a4c30','a4c4','a4c5','a4c6','a4c7','a4c8','a4c9']:
#             value_list.append('a4c')            
#         if val_idx in ['a5b','a5b0']:
#             value_list.append('a5b')            
#         if val_idx in ['a5c','a5c0','a5c1','a5c2','a5c3']:
#             value_list.append('a5c')
#         if val_idx in ['a5d','a5d0']:
#             value_list.append('a5d')
#         if val_idx in ['a6b','a6b0']:
#             value_list.append('a6b')            
#         if val_idx in ['a6c','a6c0','a6c1','a6c10','a6c11', 'a6c12','a6c13','a6c14',
#                        'a6c15','a6c16','a6c17','a6c18','a6cc19','a6c2','a6c20','a6c21',
#                        'a6c22','a6c3','a6c4','a6c5','a6c6','a6c7','a6c8','a6c9']:
#             value_list.append('a6c')             
#         if val_idx in ['a6d2','a6d20']:
#             value_list.append('a6d2')            
#         if val_idx in ['a6d3','a6d30']:
#             value_list.append('a6d3') 
#         if val_idx in ['a7b', 'a7b0','a7c','a7c0']:
#             value_list.append('a7')
#         if val_idx in ['b2a','b2a0']:
#             value_list.append('b2a') 
#         if val_idx in ['b2b','b2b0']:
#             value_list.append('b2b') 
#         if val_idx in ['c2','c20']:
#             value_list.append('c2')
#         if val_idx in ['c3a','c3a0']:
#             value_list.append('c3a')
#         if val_idx in ['c3b','c3b0','c3b1','c3b2']:
#             value_list.append('c3b')
#         if val_idx in ['d2','d20']:
#             value_list.append('d2')            
#         if val_idx in ['d3b','d3b0']:
#             value_list.append('d3b')             
#         if val_idx in ['d3c','d3c0']:
#             value_list.append('d3c')
#         if val_idx in ['d3d','d3d0']:
#             value_list.append('d3d') 
#         if val_idx in ['d4','d40']:
#             value_list.append('d4')            
#         if val_idx in ['e2','e20']:
#             value_list.append('e2')            
#         if val_idx in ['e3','e30']:
#             value_list.append('e3')
#         if val_idx in ['e4','e40']:
#             value_list.append('e4')            
#         if val_idx in ['e5','e50']:
#             value_list.append('e5')            
#         if val_idx in ['e6','e60']:
#             value_list.append('e6')
#         if val_idx in ['e7','e70']:
#             value_list.append('e7')            
#         if val_idx in ['f2','f20','f3','f30']:
#             value_list.append('f')            
            
#         else:
#             value_list.append(val_idx)
#     hybrid_results_json[key]=value_list

In [ ]:
hybrid_results_json

In [ ]:
# match_dict shows all true positives, without penalty to false categorization
match_dict= {}
for (key1, value1), (key2,value2) in zip(truth_dict.items(), hybrid_results_json.items()):
    match_count = 0
    for val_idx in value1:
        if val_idx in value2:
            match_count += 1
        match_dict[key1]=match_count/len(value1)

In [ ]:
# unpenalized accuracy of true positives
meanvals = np.mean(list(match_dict.values()))
meanvals

In [ ]:
# match_dict1 shows all true positives, with penalty to false categorization
match_dict1= {}
for (key1, value1), (key2,value2) in zip(truth_dict.items(), hybrid_results_json.items()):
    match_count = 0
    for val_idx in value2:
        if val_idx in value1:
            match_count += 1
        match_dict1[key1]=match_count/len(value2)

In [ ]:
# penalized accuracy of true positives
meanvals1 = np.mean(list(match_dict1.values()))
meanvals1